### Import Packages

In [88]:
import pandas as pd

### Load All Datasets And Check First Couple Of Rows

In [89]:
steam200kinitial = pd.read_csv('C:/Users/Joshua/Documents/git/EntityFinalProject-main/steam-200k.csv')
gamesales2017initial = pd.read_csv('C:/Users/Joshua/Documents/git/EntityFinalProject-main/Video_Game_Sales_as_of_Jan_2017.csv')

In [90]:
steam200kinitial.head(5)

,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0


In [91]:
gamesales2017initial.head(5)

,Name,Genre
0,The Sims 3,Simulation
1,World of Warcraft,Role-Playing
2,Diablo III,Role-Playing
3,Microsoft Flight Simulator,Simulation
4,StarCraft II: Wings of Liberty,Strategy


In [92]:
steam200k = steam200kinitial.rename(columns={'151603712': 'User', 'The Elder Scrolls V Skyrim': 'Name', 'purchase':'Behavior', '1.0':'Playtime'})
steam200k.drop('0', axis=1, inplace=True)
steam200k.head(5)

,User,Name,Behavior,Playtime
0,151603712,The Elder Scrolls V Skyrim,play,273.0
1,151603712,Fallout 4,purchase,1.0
2,151603712,Fallout 4,play,87.0
3,151603712,Spore,purchase,1.0
4,151603712,Spore,play,14.9


In [93]:
steam200k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User      199999 non-null  int64  
 1   Name      199999 non-null  object 
 2   Behavior  199999 non-null  object 
 3   Playtime  199999 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.1+ MB


In [94]:
steam200k.nunique()

User        12393
Name         5155
Behavior        2
Playtime     1593
dtype: int64

### Descriptive Statitics

In [95]:
steam200k.describe()

,User,Playtime
count,1.999990e+05,199999.000000
mean,1.036556e+08,17.874468
std,7.208084e+07,138.057292
min,5.250000e+03,0.100000
25%,4.738420e+07,1.000000
50%,8.691201e+07,1.000000
75%,1.542309e+08,1.300000
max,3.099031e+08,11754.000000


### Dataframe Manipulation

In [96]:
#join datasets
steam200k_genre = steam200k.merge(gamesales2017initial, how='left', on='Name')
steam200k_genre.head(5)

,User,Name,Behavior,Playtime,Genre
0,151603712,The Elder Scrolls V Skyrim,play,273.0,NaN
1,151603712,Fallout 4,purchase,1.0,Role-Playing
2,151603712,Fallout 4,play,87.0,Role-Playing
3,151603712,Spore,purchase,1.0,Strategy
4,151603712,Spore,play,14.9,Strategy


In [97]:
#remove rows where genre is NaN
steam200k_genreR=steam200k_genre.dropna()
steam200k_genreR.head(5)

,User,Name,Behavior,Playtime,Genre
1,151603712,Fallout 4,purchase,1.0,Role-Playing
2,151603712,Fallout 4,play,87.0,Role-Playing
3,151603712,Spore,purchase,1.0,Strategy
4,151603712,Spore,play,14.9,Strategy
7,151603712,Left 4 Dead 2,purchase,1.0,Shooter


In [98]:
#drop rows that have behavior of purchase because every user has purchased the game
steam200k_genreR.drop(steam200k_genreR[steam200k_genreR.Behavior == 'purchase'].index, inplace=True)

#Add new column and set 1 because every user has purchased the game
steam200k_genreR['Purchace'] = 1

#Add new column and set 0 by default
steam200k_genreR['Play'] = 0

steam200k_genreR.head(5)

C:\Python310\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Joshua\AppData\Local\Temp/ipykernel_12688/2637236718.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam200k_genreR['Purchace'] = 1
C:\Users\Joshua\AppData\Local\Temp/ipykernel_12688/2637236718.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,User,Name,Behavior,Playtime,Genre,Purchace,Play
2,151603712,Fallout 4,play,87.0,Role-Playing,1,0
4,151603712,Spore,play,14.9,Strategy,1,0
8,151603712,Left 4 Dead 2,play,8.9,Shooter,1,0
16,151603712,Left 4 Dead,play,3.3,Shooter,1,0
20,151603712,Tomb Raider,play,2.5,Action,1,0


In [99]:
#set 1 or 0 if game was played
steam200k_genreR.loc[steam200k_genreR['Playtime'] > .1, 'Play'] = 1

#drop behavior column
steam200k_genreR.drop('Behavior', axis=1, inplace=True)

steam200k_genreR.head(5)

C:\Python310\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Python310\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,User,Name,Playtime,Genre,Purchace,Play
2,151603712,Fallout 4,87.0,Role-Playing,1,1
4,151603712,Spore,14.9,Strategy,1,1
8,151603712,Left 4 Dead 2,8.9,Shooter,1,1
16,151603712,Left 4 Dead,3.3,Shooter,1,1
20,151603712,Tomb Raider,2.5,Action,1,1


In [100]:
#export data wrangeled df to file
steam200k_genreR.to_csv(r'C:\Users\Joshua\Documents\git\EntityFinalProject-main\steam200k_genreR.csv', index = False)

### Recode Dataframe For Recommender System 

In [101]:
data = steam200k_genreR

In [102]:
data2 = steam200k_genreR.Name.unique()
print(data2)

['Fallout 4' 'Spore' 'Left 4 Dead 2' 'Left 4 Dead' 'Tomb Raider'
 'BioShock Infinite' 'Grand Theft Auto IV' 'BioShock' 'Company of Heroes'
 'Grand Theft Auto V' 'Far Cry 3' 'Rocket League' 'Far Cry 4'
 'Dying Light' 'Just Cause 2' 'Portal 2' 'Tropico 4' 'Sleeping Dogs'
 'Saints Row IV' 'Half-Life 2' 'DiRT 3' 'Metro 2033'
 'Ori and the Blind Forest' 'Sniper Elite 3' 'Dishonored' 'Dead Space'
 'Portal' 'The Binding of Isaac' 'LEGO The Lord of the Rings'
 'Mass Effect' 'Borderlands 2' 'Trine' 'Far Cry 2' 'Oil Rush' 'Spintires'
 'Dead Island' 'GRID 2' 'Sonic & All-Stars Racing Transformed' 'The Crew'
 'Sonic Generations' 'Dungeon Defenders' 'TERA' 'Mafia II'
 'Europa Universalis III' 'APB Reloaded' 'Project CARS' 'World of Goo'
 'Machinarium' 'The Ball' 'Braid' 'Botanicula' 'Terraria' 'Saints Row 2'
 'GRID' 'Darksiders' 'Worms Reloaded' 'Alan Wake' 'Peggle Nights'
 'Bejeweled 3' 'The Book of Unwritten Tales' 'The Movies'
 'The Testament of Sherlock Holmes' 'Indigo Prophecy' 'DiRT 2'
 '4 El

In [103]:
for x in data2:
    data[x] = x

C:\Users\Joshua\AppData\Local\Temp/ipykernel_12688/2304973464.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[x] = x
C:\Users\Joshua\AppData\Local\Temp/ipykernel_12688/2304973464.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[x] = x


In [104]:
data = data.reset_index(drop=True)
data.head(5)

,User,Name,Playtime,Genre,Purchace,Play,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,The Whispered World,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009
0,151603712,Fallout 4,87.0,Role-Playing,1,1,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,The Whispered World,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009
1,151603712,Spore,14.9,Strategy,1,1,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,The Whispered World,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009
2,151603712,Left 4 Dead 2,8.9,Shooter,1,1,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,The Whispered World,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009
3,151603712,Left 4 Dead,3.3,Shooter,1,1,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,The Whispered World,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009
4,151603712,Tomb Raider,2.5,Action,1,1,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,The Whispered World,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009


In [105]:
data.to_csv(r'C:\Users\Joshua\Documents\git\EntityFinalProject-main\data.csv', index = False)

In [106]:
dataR= pd.read_csv('C:/Users/Joshua/Documents/git/EntityFinalProject-main/dataR.csv')

In [107]:
dataR.head(5)

,User,Name,Playtime,Genre,Purchace,Play,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009,Unnamed: 281
0,151603712,Fallout 4,87.0,Role-Playing,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,151603712,Spore,14.9,Strategy,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,151603712,Left 4 Dead 2,8.9,Shooter,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,151603712,Left 4 Dead,3.3,Shooter,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,151603712,Tomb Raider,2.5,Action,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
dataR = dataR.drop_duplicates(subset=['User'], keep='first')

In [109]:
dataR = dataR.reset_index(drop=True)
dataR.head(5)

,User,Name,Playtime,Genre,Purchace,Play,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009,Unnamed: 281
0,151603712,Fallout 4,87.0,Role-Playing,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,59945701,Company of Heroes,3.2,Strategy,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,53875128,Grand Theft Auto V,86.0,Action,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,26122540,Dungeon Defenders,6.8,Role-Playing,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,218323237,Tomb Raider,8.8,Action,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
data.head(5)

,User,Name,Playtime,Genre,Purchace,Play,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,The Whispered World,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009
0,151603712,Fallout 4,87.0,Role-Playing,1,1,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,The Whispered World,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009
1,151603712,Spore,14.9,Strategy,1,1,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,The Whispered World,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009
2,151603712,Left 4 Dead 2,8.9,Shooter,1,1,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,The Whispered World,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009
3,151603712,Left 4 Dead,3.3,Shooter,1,1,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,The Whispered World,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009
4,151603712,Tomb Raider,2.5,Action,1,1,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,The Whispered World,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009


In [111]:
#temp variables for looping through df
usergame = ''
index = 0
#loop on indexs of df
for ind in data.index:
    #if userid at ind doesnt match userid index increase index to match userid
    if data['User'][ind] != dataR['User'][index]:
        index += 1
    #check same user name, set usergame to first df game name then update the 
    #new df game name column value to 1 to show user played this game  
    if data['User'][ind] == dataR['User'][index]:
        usergame = data['Name'][ind]
        dataR[usergame][index] = data['Playtime'][ind]


C:\Users\Joshua\AppData\Local\Temp/ipykernel_12688/4234162832.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataR[usergame][index] = data['Playtime'][ind]


In [112]:
dataR.head(5)

,User,Name,Playtime,Genre,Purchace,Play,Fallout 4,Spore,Left 4 Dead 2,Left 4 Dead,...,NBA 2K11,Wildlife Park 3,NBA 2K12,Pro Evolution Soccer 2013,Breach,Championship Manager 2010,Imperial Glory,Jack Keane,Worldwide Soccer Manager 2009,Unnamed: 281
0,151603712,Fallout 4,87.0,Role-Playing,1,1,87,14,8,3,...,0,0,0,0,0,0,0,0,0,0
1,59945701,Company of Heroes,3.2,Strategy,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,53875128,Grand Theft Auto V,86.0,Action,1,1,0,0,71,1,...,0,0,0,0,0,0,0,0,0,0
3,26122540,Dungeon Defenders,6.8,Role-Playing,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,218323237,Tomb Raider,8.8,Action,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
dataR.to_csv(r'C:\Users\Joshua\Documents\git\EntityFinalProject-main\dataR2.csv', index = False)